In [65]:
import pandas as pd
import os
import re
import json
import yaml
import random
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BartForConditionalGeneration, BartConfig
from transformers import T5TokenizerFast, T5ForConditionalGeneration, T5Config
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/NanumFont/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from konlpy.tag import Kkma, Hannanum, Komoran, Okt, Mecab

In [66]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = T5TokenizerFast.from_pretrained(model_dir) 

# https://huggingface.co/paust/pko-t5-base
# transformers 의 API 를 사용하여 접근 가능합니다. 
# tokenizer 를 사용할때는 T5Tokenizer 가 아니라 T5TokenizerFast 를 사용해주십시오. 
# model 은 T5ForConditionalGeneration 를 그대로 활용하시면 됩니다.

In [67]:
config_data = {
    "general": {
        "data_path": "your_path", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
        "model_name": f"{model_dir}", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
        "output_dir": "./" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
    },
    "tokenizer": {
        "encoder_max_len": 1024,
        "decoder_max_len": 150,
        "bos_token": f"{tokenizer.bos_token}",
        "eos_token": f"{tokenizer.eos_token}",
        "unk_token": f"{tokenizer.unk_token}",
        "pad_token": f"{tokenizer.pad_token}",
        # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#',
                           '#DateOfBirth#', '#CarNumber#', '#Email#', '#CardNumber#', '#Address#', '#SSN#', '#PhoneNumber#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 50,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 4, # 50
        "per_device_eval_batch_size": 16, # 32
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 16, # 몇 개의 작은 배치를 합쳐서 큰 배치처럼 처리할지를 결정하는 파라미터: 메모리 제한 극복, 더 나은 성능
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 7,
        "fp16": True,
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "predict_with_generate": True,
        "generation_max_length": 150,
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 5, # 3
        "early_stopping_threshold": 0.0001, # 0.001
        "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
    },
    # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
    "wandb": {
        "entity": "entity_name",
        "project": "project_name",
        "name": "name"
    },
    "inference": {
        "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,
        "early_stopping": True,
        "generate_max_length": 150,
        "num_beams": 4,
        "batch_size" : 16, # 32
        # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.unk_token}", f"{tokenizer.pad_token}"]
    }
}

In [68]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 이곳에 사용자가 저장한 데이터 dir 설정하기
loaded_config['general']['data_path'] = "../../data"

In [69]:
data_path = loaded_config['general']['data_path']
train_df = pd.read_csv(os.path.join(data_path, 'train_regular-expression.csv'))
valid_df = pd.read_csv(os.path.join(data_path ,'dev_regular-expression.csv'))
test_df = pd.read_csv(os.path.join(data_path ,'test.csv'))

In [70]:
idx = random.randint(0, train_df.shape[0])
print(f'{train_df.iloc[idx, 1]}\n\n{train_df.iloc[idx, 2]}')

#Person1#: 데비, 생일에 어디에서 저녁을 먹고 싶은지 선호하는 곳이 있나요?
#Person2#: 정확히 어디로 가고 싶은지 모르겠어요. 특정 식당을 생각하는 것이 어렵네요.
#Person1#: 신문의 주말 섹션에는 훌륭한 식당 목록이 있어요.
#Person2#: 그럼, 그것을 확인해 볼 수 있겠네요.
#Person1#: 특정한 종류의 음식을 원하나요?
#Person2#: 저는 일본 음식이나 태국 음식을 정말 좋아해요.
#Person1#: 그 일본 식당인 쇼군은 좋은 리뷰를 받았어요.
#Person2#: 아, 그래요! 그 식당에 대한 리뷰를 텔레비전에서 봤어요. 리뷰어가 정말 좋아했어요!
#Person1#: 그곳이 당신의 생일에 가고 싶은 곳일까요?
#Person2#: 그게 좋을 것 같아요! 예약을 하려면 전화하는게 어떨까요?

#Person1#은 데비에게 생일 식사를 위한 식당을 물어보고, 식당 목록을 확인하는 것을 제안한다. 결국, 데비는 일본 식당 쇼군을 결정한다.


In [73]:
for idx in range(60, 90):
    print('-----'*20)
    print(train_df.iloc[idx, 1])

----------------------------------------------------------------------------------------------------
#Person1#: 제니, 즐거운 시간 보내고 있나요?
#Person2#: 네, 물론이죠. 정말 멋진 파티에 흥미로운 사람들과 맛있는 음식이 있어요.
#Person1#: 즐거워하는 모습을 보니 기쁘네요.
#Person2#: 초대해주셔서 감사합니다.
#Person1#: 별말씀을요. 샴페인 한 잔 더 드릴까요?
#Person2#: 네, 한 잔 더 좋아요. 정말 훌륭한 주인이세요. 모든 것에 감사드립니다.
#Person1#: 당신을 여기 모시게 되어 기뻐요.
----------------------------------------------------------------------------------------------------
#Person1#: 오, 린다. 미국에 공부하러 가는 것에 대해 정말 신나겠어요. 
#Person2#: 오, 그래요. 저는 항상 미국에 가고 싶었어요. 새로운 사람들을 만나고 새로운 친구들을 사귀는 것을 좋아해요.
#Person1#: 그럼, 저는 분명히 그럴 것이라고 확신해요.
#Person2#: 그런데, 한 가지 걱정이 있어요.
#Person1#: 무슨 문제가 있나요?
#Person2#: 제 주인 가족에 대해 조금 걱정이 되어요. 알다시피, 계약서에 따르면 저는 그들을 위해 요리를 해야 해요.
#Person1#: 그래요?
#Person2#: 그런데, 제가 중국식 요리를 하면 그들이 좋아하지 않을까봐 두려워요.
#Person1#: 오, 들어봐요, 당신은 곧 배울 거에요, 그리고 그들이 어떻게 준비하길 원하는지 분명히 설명해 줄 거에요.
#Person2#: 오, 그럴 수 있기를 바라요.
-------------------------------------------------------------------------------------------------

- 요약문을 보면 -ㅂ니다. -ㅆ다. -ㄴ다. -ㅅ이다.
- 맞춰줘야 할 것 같음!

In [40]:
train_summary = train_df['summary'].tolist()

In [41]:
print(len(re.search(r'\.[A-Za-z가-힣0-9]+', '#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.5b').group()))

3


In [42]:
def summary_search(pattern, summary):
    for idx in tqdm(range(0, len(summary))):
        search = re.search(f'{pattern}', summary[idx])
        if search:
            cond = search.group()
            if len(cond) > 1:
                print('-----'*20)
                print(idx, cond)
                print(summary[idx])

In [43]:
pattern = r'\다\.[A-Za-z가-힣0-9]+'
summary_search(pattern, train_summary)

100%|██████████| 12457/12457 [00:00<00:00, 764413.76it/s]

----------------------------------------------------------------------------------------------------
10476 다.4
#Person2#가 잘못된 방향으로 운전하자 #Person1#은 여기서 걸어가기로 결정합니다. #Person2#가 U턴을 제안하지만, #Person1#은 교통 체증 때문에 거절합니다.4
----------------------------------------------------------------------------------------------------
10488 다.폴리는
닉은 폴리에게 켈리 여사의 강연을 준비하도록 요청합니다.폴리는 화요일에 그녀의 수업을 진행할 것을 제안합니다. 닉은 이 박사님과 이야기할 것이고 폴리에게 학생들에게 변동 사항을 알리도록 요청합니다.
----------------------------------------------------------------------------------------------------
10546 다.레슬리는
레슬리는 레슬리에게 관심 있는 프로그램을 보는 것을 선호합니다.레슬리는 텔레비전이 시간 낭비가 될 수 있다고 믿습니다. 또한 레슬리는 사람이 다른 외부 관심사가 있다면, 그는 텔레비전 프로그램에 대해 더 신중하게 생각할 것이라고 추측합니다. 레슬리는 텔레비전이 사람들이 자신들을 즐겁게 하는 능력을 위협한다고 생각합니다.


In [44]:
train_summary[10476] = re.sub(pattern, '다.', train_summary[10476])
train_summary[10488] = re.sub(pattern, '다. 폴리는', train_summary[10488])
train_summary[10546] = re.sub(pattern, '다. 레슬리는', train_summary[10546])

print(train_summary[10546])

레슬리는 레슬리에게 관심 있는 프로그램을 보는 것을 선호합니다. 레슬리는 텔레비전이 시간 낭비가 될 수 있다고 믿습니다. 또한 레슬리는 사람이 다른 외부 관심사가 있다면, 그는 텔레비전 프로그램에 대해 더 신중하게 생각할 것이라고 추측합니다. 레슬리는 텔레비전이 사람들이 자신들을 즐겁게 하는 능력을 위협한다고 생각합니다.


In [45]:
pattern = r'다다'
summary_search(pattern, train_summary)

100%|██████████| 12457/12457 [00:00<00:00, 630631.43it/s]

----------------------------------------------------------------------------------------------------
2086 다다
#Person2#는 차가 시동이 걸리지 않아 늦었고 #Person1#에게 연락할 수가 없었습니다. #Person2#는 #Person1#에게 서비스를 복구하기 위해 요금을 내는 것을 제안합니다다.
----------------------------------------------------------------------------------------------------
2105 다다
#Person1#은 몸매가 좋은 여자를 좋아하고, #Person2#는 성격이 좋은 여자를 좋아합니다다. 헨리는 여자를 좋아하지 않습니다.
----------------------------------------------------------------------------------------------------
4995 다다
#Person1#과 고든은 맛있는 식사를 하고 나서 떠나려고 한다. 고든은 계산을 다 내고 싶어하지만 #Person1#은 계산을 각자하자고 주장한다. 결국 고든은 계산을 각자 결제하는 것에 동의한다다.
----------------------------------------------------------------------------------------------------
5028 다다
#Person2#의 차례가 왔지만 #Person2#는 준비를 잊어버렸다다. 선생님은 #Person2#에게 다음에 하도록 요청한다.


In [46]:
train_summary[2086] = re.sub(pattern, '다', train_summary[2086])
train_summary[2105] = re.sub(pattern, '다', train_summary[2105])
train_summary[4995] = re.sub(pattern, '다', train_summary[4995])
train_summary[5028] = re.sub(pattern, '다', train_summary[5028])

print(train_summary[5028])

#Person2#의 차례가 왔지만 #Person2#는 준비를 잊어버렸다. 선생님은 #Person2#에게 다음에 하도록 요청한다.


In [47]:
pattern = r'\.\.'
summary_search(pattern, train_summary)

100%|██████████| 12457/12457 [00:00<00:00, 879514.61it/s]

----------------------------------------------------------------------------------------------------
1810 ..
#Person2#는 #Person1#에게 사람들이 자신의 서버를 사지 않고도 웹사이트를 운영할 수 있게 해주는 웹 호스팅 회사에서 일하고 싶다고 말합니다. 급여는 합리적이고 복리후생도 좋지만, 이 일자리가 얼마나 오래 지속될 수 있을지는 확신하지 못 합니다.. #Person1#는 #Person2#에게 학교로 돌아가서 직업 기술을 향상시킬 것을 제안하며, 더 많은 훈련이 더 좋은 일자리를 찾는 데 도움이 될 수 있다고 말합니다. 하지만 #Person2#는 학교로 돌아가기 위한 자금이 없습니다.
----------------------------------------------------------------------------------------------------
4986 ..
#Person2#는 동생을 위해 미디움 사이즈의 노란색 코트를 사려 했지만, 방금 막 품절되었다.. #Person1#는 10% 할인된 녹색 코트를 추천한다.


In [48]:
train_summary[1810] = re.sub(pattern, '.', train_summary[1810])
train_summary[4986] = re.sub(pattern, '.', train_summary[4986])

print(train_summary[4986])

#Person2#는 동생을 위해 미디움 사이즈의 노란색 코트를 사려 했지만, 방금 막 품절되었다. #Person1#는 10% 할인된 녹색 코트를 추천한다.


In [49]:
temp = []
for summary in train_summary:
    words = summary.split()
    for word in words:
        if '다.' in word:
            temp.append(word)

print(len(temp))
temp = list(set(temp))
print(len(temp))
print(temp)

22716
2004
['준비합니다.', '입금하였습니다.', '사람이되었다.', '불렀습니다.', '청하였습니다.', '물어봤습니다.', '간식이다.', '생각한다.', '교환합니다.', '상했다.', '깨닫는다.', '그립다.', '대접했다.', '잡아주었습니다.', '마무리한다.', '학생이다.', '작성한다.', '없어졌다.', '버렸다.', '예약해준다.', '태워준다.', '만족해한다.', '반품한다.', '확입합니다.', '확정합니다.', '내준다.', '사줍니다.', '친구입니다.', '나쁘다.', '쓴다.', '선택한다.', '개설한다.', '받습니다.', '회상합니다.', '갈계획이다.', '시간이다.', '상담해줍니다.', '동의했습니다.', '만납니다.', '배송됩니다.', '구입한다.', '요청합니다.', '들어있습니다.', '도난당했습니다.', '도와줬습니다.', '서두른다.', '얻었습니다.', '표시합니다.', '찾아갑니다.', '비관적이다.', '팔았다.', '선호합니다.', '구입했습니다.', '변덕스럽습니다.', '해소합니다.', '사용했다.', '비난합니다.', '사준다.', '커맨더입니다.', '표현한다.', '제안해줍니다.', '면접했다.', '검소합니다.', '안심시킵니다.', '들었습니다.', '축하하였습니다.', '가지게된다.', '허락한다.', '허용합니다.', '칭찬합니다.', '잔다.', '성공적이었다.', '얹는다.', '환불한다.', '사용합니다.', '깨운다.', '답합니다.', '제안하였습니다.', '실패한다.', '키웠다.', '청구됩니다.', '선물입니다.', '끼었다.', '작가입니다.', '끝났습니다.', '의심합니다.', '열어놓았다.', '끈다.', '나아졌다.', '씁니다.', '감사해한다.', '놀립니다.', '즐깁니다.', '했습니다.', '같다.', '열었다.', '잡았다.', '오염입니다.', '바뀐다.', '고른다.', '축하합니다.', '끔찍했다.', '남깁니다.', '심해집니다.'

In [50]:
for t in temp:
    if re.search(r'\다\.[A-Za-z가-힣0-9]+', t):
        print(t)

In [51]:
for t in temp:
    if re.search(r'다다', t):
        print(t)

In [52]:
for t in temp:
    if re.search(r'\.\.', t):
        print(t)

In [23]:
kkma = Kkma()
hannanum = Hannanum()
komoran = Komoran()
okt = Okt()
mecab = Mecab()

In [58]:
def make_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

make_folder('./kkma')
make_folder('./mecab')

In [24]:
kkma_morphs = []
for idx in tqdm(range(0, len(temp))):
    t = kkma.morphs(temp[idx])
    kkma_morphs.append(t)
print(kkma_morphs)

filename = './kkma/train_kkma_morphs.txt'
dictn = {key: value for key, value in zip(temp, kkma_morphs)}
with open(filename, 'w', encoding='utf-8') as file:
    for key, value in dictn.items():
        file.write(f'{key}: {value}\n')

100%|██████████| 2004/2004 [00:13<00:00, 145.06it/s]

[['준비', '하', 'ㅂ니다', '.'], ['입금', '하', '였', '습니다', '.'], ['사람', '이', '되', '었', '다', '.'], ['불르', '었', '습니다', '.'], ['청하', '이', '었', '습니다', '.'], ['물어보', '았', '습니다', '.'], ['간식', '이', '다', '.'], ['생각', '하', 'ㄴ다', '.'], ['교환', '하', 'ㅂ니다', '.'], ['상하', '었', '다', '.'], ['깨닫', '는', '다', '.'], ['그립', '다', '.'], ['대접', '하', '었', '다', '.'], ['잡', '아', '주', '었', '습니다', '.'], ['마무리', '하', 'ㄴ다', '.'], ['학생', '이', '다', '.'], ['작성', '하', 'ㄴ다', '.'], ['없', '어', '지', '었', '다', '.'], ['버리', '었', '다', '.'], ['예약', '하', '어', '주', 'ㄴ다', '.'], ['태우', '어', '주', 'ㄴ다', '.'], ['만족', '하', '어', '하', 'ㄴ다', '.'], ['반품', '하', 'ㄴ다', '.'], ['확', '입하', 'ㅂ니다', '.'], ['확정', '하', 'ㅂ니다', '.'], ['내주', 'ㄴ다', '.'], ['사', '아', '주', 'ㅂ니다', '.'], ['친구', '이', 'ㅂ니다', '.'], ['나쁘', '다', '.'], ['쓰', 'ㄴ다', '.'], ['선택', '하', 'ㄴ다', '.'], ['개설', '하', 'ㄴ다', '.'], ['받', '습니다', '.'], ['회상', '하', 'ㅂ니다', '.'], ['갈', 'ㄹ', '계획', '이', '다', '.'], ['시간', '이', '다', '.'], ['상담', '하', '어', '주', 'ㅂ니다', '.'], ['동의', '하', '었', '습니다', '.'], ['만나', 'ㅂ니다'

In [26]:
kkma_morphs2 = []
for idx in tqdm(range(0, len(kkma_morphs))):
    if kkma_morphs[idx][-1] == '.':
        cond = kkma_morphs[idx][-2] # . 앞에 ~다 수집
    kkma_morphs2.append(cond)
print(kkma_morphs2)

100%|██████████| 2004/2004 [00:00<00:00, 1345076.85it/s]

['ㅂ니다', '습니다', '다', '습니다', '습니다', '습니다', '다', 'ㄴ다', 'ㅂ니다', '다', '다', '다', '다', '습니다', 'ㄴ다', '다', 'ㄴ다', '다', '다', 'ㄴ다', 'ㄴ다', 'ㄴ다', 'ㄴ다', 'ㅂ니다', 'ㅂ니다', 'ㄴ다', 'ㅂ니다', 'ㅂ니다', '다', 'ㄴ다', 'ㄴ다', 'ㄴ다', '습니다', 'ㅂ니다', '다', '다', 'ㅂ니다', '습니다', 'ㅂ니다', 'ㅂ니다', 'ㄴ다', 'ㅂ니다', '습니다', '습니다', '습니다', 'ㄴ다', '습니다', 'ㅂ니다', 'ㅂ니다', '다', '다', 'ㅂ니다', '습니다', '습니다', 'ㅂ니다', '다', 'ㅂ니다', 'ㄴ다', 'ㅂ니다', 'ㄴ다', 'ㅂ니다', '다', 'ㅂ니다', 'ㅂ니다', '습니다', '습니다', 'ㄴ다', 'ㄴ다', 'ㅂ니다', 'ㅂ니다', 'ㄴ다', '다', '다', 'ㄴ다', 'ㅂ니다', 'ㄴ다', 'ㅂ니다', '습니다', 'ㄴ다', '다', 'ㅂ니다', 'ㅂ니다', '다', 'ㅂ니다', '습니다', 'ㅂ니다', '다', 'ㄴ다', '다', 'ㅂ니다', 'ㄴ다', 'ㅂ니다', 'ㅂ니다', '습니다', '다', '다', '다', 'ㅂ니다', 'ㄴ다', 'ㄴ다', 'ㅂ니다', '다', 'ㅂ니다', 'ㅂ니다', '습니다', '습니다', 'ㄴ다', 'ㅂ니다', 'ㄴ다', 'ㅂ니다', '습니다', 'ㅂ니다', '습니다', '다', '다', 'ㅂ니다', 'ㄴ다', 'ㅂ니다', 'ㄴ다', 'ㄴ다', '습니다', 'ㄴ다', '다', '습니다', '다', 'ㄴ다', '습니다', 'ㄴ다', 'ㄴ다', '다', '습니다', '습니다', 'ㅂ니다', 'ㅂ니다', '다', '다', '습니다', 'ㅂ니다', 'ㄴ다', 'ㄴ다', 'ㅂ니다', '습니다', '습니다', 'ㄴ다', 'ㄴ다', '습니다', '습니다', 'ㄴ다', 'ㅂ니다', 'ㅂ니다', '다', 'ㅂ니다', 'ㄴ다', '다', 'ㄴ다', 'ㄴ다', 'ㅂ니다', '다', '습니다', 

In [27]:
end_text = kkma_morphs2
end_text = list(set(end_text))
print(len(end_text), end_text)

10 ['ㅂ니다', '이다', '옵니다', 'ㄴ다', 'ㄹ란다', '픕다', '하다', '르렀다', '습니다', '다']


In [28]:
dictn = {}
with open('./kkma/train_kkma_morphs.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if ': ' in line:
            key, value = line.split(': ', 1)
            dictn[key.strip()] = eval(value.strip())

In [29]:
conds = ['이다', 'ㄴ다', 'ㄹ란다', '픕다', '하다', '르렀다', '다'] # ['옵니다', '습니다', 'ㅂ니다']

In [30]:
check_summary_list = []
for j in tqdm(range(0, train_df.shape[0])):
    morphs = kkma.morphs(train_summary[j])
    check_summary_list.append(morphs)

100%|██████████| 12457/12457 [05:14<00:00, 39.60it/s]


In [31]:
for i in tqdm(range(0, len(conds))):
    idxs = []
    sums = []    
    for j in range(0, train_df.shape[0]):
        if conds[i] in check_summary_list[j]:    
            # print(j, train_summary[j])
            idxs.append(j)
            sums.append(train_summary[j])
    temp = pd.DataFrame({
        'fname': idxs,
        'summary': sums
    })
    temp.to_csv(f'./kkma/train_kkma_{conds[i]}.csv', index=False)

100%|██████████| 7/7 [00:00<00:00, 68.32it/s]


In [53]:
mecab_morphs = []
for idx in tqdm(range(0, len(temp))):
    t = mecab.morphs(temp[idx])
    mecab_morphs.append(t)
print(mecab_morphs)

filename = './mecab/train_mecab_morphs.txt'
dictn = {key: value for key, value in zip(temp, mecab_morphs)}
with open(filename, 'w', encoding='utf-8') as file:
    for key, value in dictn.items():
        file.write(f'{key}: {value}\n')

100%|██████████| 2004/2004 [00:00<00:00, 54395.69it/s]

[['준비', '합니다', '.'], ['입금', '하', '였', '습니다', '.'], ['사람', '이', '되', '었', '다', '.'], ['불렀', '습니다', '.'], ['청하', '였', '습니다', '.'], ['물', '어', '봤', '습니다', '.'], ['간식', '이', '다', '.'], ['생각', '한다', '.'], ['교환', '합니다', '.'], ['상했', '다', '.'], ['깨닫', '는다', '.'], ['그립', '다', '.'], ['대접', '했', '다', '.'], ['잡', '아', '주', '었', '습니다', '.'], ['마무리', '한다', '.'], ['학생', '이', '다', '.'], ['작성', '한다', '.'], ['없', '어', '졌', '다', '.'], ['버렸', '다', '.'], ['예약', '해', '준다', '.'], ['태워', '준다', '.'], ['만족', '해', '한다', '.'], ['반품', '한다', '.'], ['확', '입', '합니다', '.'], ['확정', '합니다', '.'], ['내준다', '.'], ['사', '줍니다', '.'], ['친구', '입니다', '.'], ['나쁘', '다', '.'], ['쓴다', '.'], ['선택', '한다', '.'], ['개설', '한다', '.'], ['받', '습니다', '.'], ['회상', '합니다', '.'], ['갈', '계획', '이', '다', '.'], ['시간', '이', '다', '.'], ['상담', '해줍니다', '.'], ['동의', '했', '습니다', '.'], ['만납니다', '.'], ['배송', '됩니다', '.'], ['구입', '한다', '.'], ['요청', '합니다', '.'], ['들', '어', '있', '습니다', '.'], ['도난', '당했', '습니다', '.'], ['도와', '줬', '습니다', '.'], ['서두른다', '.'], ['얻'

In [54]:
mecab_morphs2 = []
for idx in tqdm(range(0, len(mecab_morphs))):
    if mecab_morphs[idx][-1] == '.':
        cond = mecab_morphs[idx][-2] # . 앞에 ~다 수집
    mecab_morphs2.append(cond)
print(mecab_morphs2)

100%|██████████| 2004/2004 [00:00<00:00, 1311906.54it/s]

['합니다', '습니다', '다', '습니다', '습니다', '습니다', '다', '한다', '합니다', '다', '는다', '다', '다', '습니다', '한다', '다', '한다', '다', '다', '준다', '준다', '한다', '한다', '합니다', '합니다', '내준다', '줍니다', '입니다', '다', '쓴다', '한다', '한다', '습니다', '합니다', '다', '다', '해줍니다', '습니다', '만납니다', '됩니다', '한다', '합니다', '습니다', '습니다', '습니다', '서두른다', '습니다', '합니다', '찾아갑니다', '다', '다', '합니다', '습니다', '습니다', '합니다', '다', '합니다', '준다', '입니다', '한다', '해줍니다', '다', '합니다', '시킵니다', '습니다', '습니다', '된다', '한다', '합니다', '합니다', '잔다', '다', '는다', '한다', '합니다', '깨운다', '합니다', '습니다', '한다', '다', '됩니다', '입니다', '다', '입니다', '습니다', '합니다', '다', '끈다', '다', '씁니다', '해한다', '다', '즐깁니다', '습니다', '다', '다', '다', '입니다', '바뀐다', '고른다', '합니다', '다', '깁니다', '심해집니다', '습니다', '습니다', '준다', '입니다', '건다', '합니다', '습니다', '줍니다', '습니다', '다', '다', '집니다', '한다', '니다', '한다', '받아들인다', '습니다', '한다', '다', '습니다', '다', '한다', '습니다', '들려준다', '한다', '다', '습니다', '습니다', '합니다', '입니다', '다', '다', '습니다', '랍니다', '놀려댄다', '된다', '합니다', '습니다', '습니다', '한다', '산다', '습니다', '습니다', '한다', '나갑니다', '니다', '다', '줍니다', '해진다', '다', '다', '원한

In [55]:
end_text = mecab_morphs2
end_text = list(set(end_text))
print(len(end_text), end_text)

251 ['걸어갑니다', '만납니다', '원한다', '가집니다', '남긴다', '짠다', '찾아온다', '붙입니다', '세운다', '들려준다', '빈다', '들어준다', '열린다', '배웁니다', '다닙니다', '큽니다', '잊어버린다', '습니다', '집니다', '뛴다', '우긴다', '깁니다', '두려워한다', '걸린다', '가져온다', '는다', '쓴다', '즐긴다', '대한다', '알아본다', '쉰다', '마칩니다', '구한다', '받아들여진다', '해본다', '다', '나옵니다', '내린다', '냅니다', '말린다', '가르칩니다', '심합니다', '살펴봅니다', '일어납니다', '해집니다', '던진다', '떨린다', '시킨다', '보인다', '다툰다', '삽니다', '떠난다', '나온다', '랍니다', '전해진다', '놀린다', '향한다', '찾아갑니다', '갑니다', '돌려준다', '일으킵니다', '합니다', '니다', '한다', '받아들인다', '놀려댄다', '만듭니다', '씁니다', '세웁니다', '알아차립니다', '건다', '울린다', '바랍니다', '빠진다', '내줍니다', '부러워한다', '들어섭니다', '본다', '보낸다', '알아차린다', '생깁니다', '느낍니다', '찾아간다', '괴롭힌다', '버립니다', '납니다', '당합니다', '지낸다', '해합니다', '깨어납니다', '따릅니다', '자른다', '고른다', '나갑니다', '낸다', '잔다', '마신다', '돌아온다', '기쁩니다', '나타냅니다', '봅니다', '해한다', '어울립니다', '기다립니다', '당한다', '풀린다', '끕니다', '바뀐다', '듭니다', '드립니다', '밝혀집니다', '헤어집니다', '픕다', '놀라워한다', '된다', '중입니다', '나눈다', '바쁩니다', '청한다', '마주친다', '봐준다', '가져간다', '일어난다', '물어본다', '향합니다', '깨운다', '돌아간다', '보여준다', '받아들여집니다', '놔둔다', '여긴다', '춥니다

In [59]:
dictn = {}
with open('./mecab/train_mecab_morphs.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if ': ' in line:
            key, value = line.split(': ', 1)
            dictn[key.strip()] = eval(value.strip())

In [60]:
conds = ['걸어갑니다', '만납니다', '원한다', '가집니다', '남긴다', '짠다', '찾아온다', '붙입니다', '세운다', '들려준다', '빈다', '들어준다', '열린다', '배웁니다', '다닙니다', '큽니다', '잊어버린다', '습니다', '집니다', '뛴다', '우긴다', '깁니다', '두려워한다', '걸린다', '가져온다', '는다', '쓴다', '즐긴다', '대한다', '알아본다', '쉰다', '마칩니다', '구한다', '받아들여진다', '해본다', '다', '나옵니다', '내린다', '냅니다', '말린다', '가르칩니다', '심합니다', '살펴봅니다', '일어납니다', '해집니다', '던진다', '떨린다', '시킨다', '보인다', '다툰다', '삽니다', '떠난다', '나온다', '랍니다', '전해진다', '놀린다', '향한다', '찾아갑니다', '갑니다', '돌려준다', '일으킵니다', '합니다', '니다', '한다', '받아들인다', '놀려댄다', '만듭니다', '씁니다', '세웁니다', '알아차립니다', '건다', '울린다', '바랍니다', '빠진다', '내줍니다', '부러워한다', '들어섭니다', '본다', '보낸다', '알아차린다', '생깁니다', '느낍니다', '찾아간다', '괴롭힌다', '버립니다', '납니다', '당합니다', '지낸다', '해합니다', '깨어납니다', '따릅니다', '자른다', '고른다', '나갑니다', '낸다', '잔다', '마신다', '돌아온다', '기쁩니다', '나타냅니다', '봅니다', '해한다', '어울립니다', '기다립니다', '당한다', '풀린다', '끕니다', '바뀐다', '듭니다', '드립니다', '밝혀집니다', '헤어집니다', '픕다', '놀라워한다', '된다', '중입니다', '나눈다', '바쁩니다', '청한다', '마주친다', '봐준다', '가져간다', '일어난다', '물어본다', '향합니다', '깨운다', '돌아간다', '보여준다', '받아들여집니다', '놔둔다', '여긴다', '춥니다', '느낀다', '나간다', '고마워한다', '부러워합니다', '돌려줍니다', '사옵니다', '채워준다', '밝힌다', '렀다', '알려준다', '들어옵니다', '시킵니다', '기다린다', '가져갑니다', '입니다', '돌아옵니다', '나눕니다', '찾아본다', '돌아갑니다', '아쉬워합니다', '뺍니다', '비쌉니다', '보냅니다', '다릅니다', '넘깁니다', '부딪힌다', '줄인다', '내준다', '이긴다', '줍니다', '해줍니다', '둔다', '버린다', '알린다', '번다', '느껴집니다', '들른다', '넘어선다', '부른다', '자릅니다', '다닌다', '걸립니다', '망설인다', '겁니다', '간다', '느껴진다', '덧붙인다', '해진다', '놀란다', '떠올린다', '구합니다', '난다', '못한다', '꿉니다', '무서워한다', '넘친다', '끌어들인다', '데려갑니다', '든다', '바란다', '산다', '서두른다', '빌린다', '일으킨다', '픕니다', '배운다', '취합니다', '들어갑니다', '겹칩니다', '잃어버립니다', '내립니다', '옵니다', '알아낸다', '맡깁니다', '정한다', '태웁니다', '넘어갑니다', '나타납니다', '남겨준다', '스러워한다', '뿌린다', '안다', '바꾼다', '보입니다', '탄다', '이릅니다', '친다', '살아간다', '밝힙니다', '달린다', '가르친다', '끈다', '온다', '비운다', '뜹니다', '따른다', '부릅니다', '심해집니다', '태운다', '이른다', '모릅니다', '들려줍니다', '받아들입니다', '건넵니다', '그리워진다', '응한다', '마십니다', '물어봅니다', '진다', '겹친다', '즐깁니다', '떠납니다', '떨어뜨린다', '모른다', '됩니다', '칩니다', '준다', '만난다', '달립니다']

In [61]:
check_summary_list = []
for j in tqdm(range(0, train_df.shape[0])):
    morphs = mecab.morphs(train_summary[j])
    check_summary_list.append(morphs)

100%|██████████| 12457/12457 [00:01<00:00, 11621.64it/s]


In [62]:
for i in tqdm(range(0, len(conds))):
    idxs = []
    sums = []    
    for j in range(0, train_df.shape[0]):
        if conds[i] in check_summary_list[j]:    
            # print(j, train_summary[j])
            idxs.append(j)
            sums.append(train_summary[j])
    temp = pd.DataFrame({
        'fname': idxs,
        'summary': sums
    })
    temp.to_csv(f'./mecab/train_mecab_{conds[i]}.csv', index=False)

100%|██████████| 251/251 [00:02<00:00, 119.61it/s]


---

In [ ]:
df = pd.read_csv('./ㅂ니다_train.csv')
cond = 500
for idx in range(1010+cond, 1110+cond):
    print(df.iloc[idx, 0], df.iloc[idx, 1])

너는 'ㅂ니다.'와 같은 종결어미를 'ㄴ다.' 등과 같이 가볍게 바꿔야 돼.

몇 개의 예시를 줄게. 예시를 보고 내가 제공한 데이터를 적절히 바꿔줘.

[예시]

원본:

변환:


[데이터]

원본:

변환:
